In [1]:
import numpy as np
import pandas as pd
import seaborn as sns; sns.set()
from matplotlib import pyplot as plt, cm, colors
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d import Axes3D
import shutil


sns.set_style("ticks")
sns.set_style("ticks")
sns.despine()
from matplotlib import pyplot as plt, cm, colors
from tqdm.notebook import tqdm

import glob
import os.path as ospath
import os
import pickle
import re
from sys import executable
from subprocess import check_output
from PyQt5.QtWidgets import QFileDialog, QApplication
from IPython.display import HTML

from scipy import optimize
from scipy.spatial import distance
from scipy import linalg
from scipy import signal
from scipy import stats
from sklearn.cluster import MeanShift, estimate_bandwidth

from picasso.picasso import io
from picasso.picasso.postprocess import link, compute_dark_times
from picasso.picasso.render import render
from picasso.picasso.gui.render import estimate_kinetic_rate, fit_cum_exp
from picasso.picasso.lib import append_to_rec

<Figure size 640x480 with 0 Axes>

In [2]:
def load_files(dirname):
    
    os.chdir(dirname)
    files = glob.glob("*.hdf5")
    
    if files:
        print("{} HDF5 files found.".format(len(files)))
    else:
        print("No HDF5 files found at: {}".format(dirname))
            
    return files

def load_arc_data_df(dirname, filename):
    file = ospath.join(dirname, filename)

    try: 
        df = pd.read_pickle(file)
    except FileNotFoundError:
        print("No results of previously analyzed datasets were detected.")
        return None
    else: 
        print("Results of previously analyzed datasets were detected.")
        return df
    
def identify_new_files(files, df_ring_data):
    """
    Identify which files have already been analyzed previously. 
    Return list of new files for processing
    """
    new_files = []
    for file in files:
        if file not in df_ring_data['filename'].values:
            new_files.append(file)
    
    n_old = len(files)-len(new_files)
    n_new = len(new_files)
    if n_old == 1:
        print(" {} HDF5 file was previously analyzed.".format(n_old))
    else: 
        print(" {} HDF5 files were previously analyzed.".format(n_old))
        
    if n_new == 1:
        print(" {} HDF5 file is new and will be analyzed.".format(n_new))
    else: 
        print(" {} HDF5 files are new and will be analyzed.".format(n_new))
        
    return new_files

def identify_fov_cell_type(df_ring_data, filenames, fov_id_start):
    """
    For each file identify 
    - cell type: sporulating or vegetative cells
    - fov index: fov from which the picks were generated
    Results will be saved in a dataframe with columns:
    'fov_id', 'filename', 'cell_type'
    
    If some files have been analyzed before the current script execution 
    the results were saved in the ring_data file and loaded to df_ring_data.
    (columns: "fov_id", "cell_type", "filename", "group", ... where 
    group are the pick ids.)
    If no prior analysis results exist, then df_ring_data = None. 
    Therefore we can check if a new file belongs to a previously analyzed fov.
    
    """
    dictionary = {}
    filenames_no_cell_type = []
    
    fov_id_counter = fov_id_start + 1
    
    for i, filename in enumerate(filenames):
        # cell type: spor or veg:
        cell_type = np.nan
                
        spor_found = re.search('spor', filename, re.IGNORECASE)
        veg_found = re.search('veg', filename, re.IGNORECASE)
        
        if spor_found and not veg_found:
            cell_type = 'spor'
        elif not spor_found and veg_found:
            cell_type = 'veg'
        elif spor_found and veg_found:
            # consider the string occuring first as cell type determining string
            spor_found_location = spor_found.start()
            veg_found_location = veg_found.start()
            
            if spor_found_location < veg_found_location:
                cell_type = 'spor'
            else:
                cell_type = 'veg'
        else:
            cell_type = np.nan
            filenames_no_cell_type.append(filename)
            

        
        # search if a file from the same fov was already registered:
        if not pd.isnull(cell_type):
            # get substrings of filename that do not contain the cell_type string.
            filename_substrings = re.split(cell_type, filename, flags = re.IGNORECASE)
            
            
            
            # check if an already registered file exists that contains the substrings
            filenames_found = []
            old_or_new_file = [] # True if file from previous run of the script, False if new file.
            for filename2 in dictionary.keys(): # search in files that were already registered.
                contained = all([substring in filename2 for substring in filename_substrings])
                if contained and filename != filename2:
                    filenames_found.append(filename2)
                    old_or_new_file.append(False)
            if isinstance(df_ring_data, pd.DataFrame):
                previously_analyzed_files = np.unique(df_ring_data['filename'])
                for filename2 in previously_analyzed_files:
                    contained = all([substring in filename2 for substring in filename_substrings])
                    if contained and filename != filename2:
                        filenames_found.append(filename2)
                        old_or_new_file.append(True)
            
            # if one other file was found: assign the existing fov index to the newly registred file
            # if no other file was found: assign a new fov index to the newly registered file
            if len(filenames_found) > 1:
                raise Exception('''Files from the same FOV than ''' + filename + ''' where searched. 
                However more than one other file was detected: 
                ''' + '\n'.join(filenames_found))
            elif len(filenames_found) == 1:
                filename_found = filenames_found[0]
                if old_or_new_file[0]: # filename_found is from previous run of the script
                    fov_id = df_ring_data.loc[df_ring_data['filename'] == filename_found, 'fov_id'].iloc[0]
                if not old_or_new_file[0]: # filename_found is also a new file.
                    fov_id = dictionary[filename_found][0]
            else: # No file from the same fov previously registered
                fov_id = fov_id_counter
                fov_id_counter += 1
        
        else:
            fov_id = np.nan
            
        dictionary[filename] = [fov_id, filename, cell_type]
        
    df_results = pd.DataFrame.from_dict(dictionary, orient = 'index', columns = ['fov_id', 'filename', 'cell_type'])
    df_results = df_results.reset_index()

    print()
    print('The cell type (spr or veg) of these files could not be determined')
    print('and thus cannot be used for further analysis:')
    for filename in filenames_no_cell_type:
        print(' -', filename)
        
    return df_results


def load_data(path):

    try:
        locs, info = io.load_locs(path)
    except io.NoMetadataFileError:
        return None, None, None
    
    try:
        pixelsize = info[1]["Pixelsize"]
    except:  
        print("No pixelsize found in yaml file. Default 130 nm used.")
  
    if hasattr(locs, "x_pick_rot"):

        # convert px to nm
        locs.x *= pixelsize
        locs.y *= pixelsize
        locs.x_pick_rot *= pixelsize
        locs.y_pick_rot *= pixelsize
        
        return locs, info, pixelsize
    else:
        print('x_pick_rot column is missing!')
        return None, None, None

def double_gaus(x,a,x0,sigma, b, x1, sigma1):
    return a*np.exp(-(x-x0)**2/(2*sigma**2)) + b*np.exp(-(x-x1)**2/(2*sigma1**2))

def gaus(x,a,x0,sigma):
    return a*np.exp(-(x-x0)**2/(2*sigma**2)) 

def histogram(data, binning=100, column = "y"):
    # histogram
    n, bins = np.histogram(data[column], bins=binning)
    centers = (bins[:-1] + bins[1:]) / 2
    hist_data = [n, centers]
    return hist_data

def fit_peaks(data, p0, binning=100, column = "y", hist_data = None):
    if hist_data is None:
        hist_data = histogram(data, binning, column)
    
    n = hist_data[0]
    centers = hist_data[1]
    
    try:
        p_fit, p_cov = optimize.curve_fit(double_gaus, centers, n, p0=p0)
    except:
        p_fit = [0,0,0,0,0,0]
        
    p_fit[2] = np.abs(p_fit[2])
    p_fit[5] = np.abs(p_fit[5])
    
    return p_fit, hist_data

def fit_peak(data, p0, binning=100, column = "y", hist_data = None):
    if hist_data is None:
        hist_data = histogram(data, binning, column)
    
    n = hist_data[0]
    centers = hist_data[1]
    
    try:
        p_fit, p_cov = optimize.curve_fit(gaus, centers, n, p0=p0)
    except:
        p_fit = np.array([0,0,0])
        
    p_fit[2] = np.abs(p_fit[2])
    
    return p_fit, hist_data

def find_peaks(data, binning=100, axes="y"):
    
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
    elif axes == "xyz":
        column = 2
    elif axes == "z":
        column = "z"
    
    # find peaks
    
    bandwidth = estimate_bandwidth(data[column].reshape(-1, 1), quantile=0.2, n_samples=binning)
    #print("estimated bandwidth: "+str(bandwidth))
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(data[column].reshape(-1, 1))
    labels = ms.labels_
    peaks = np.sort(ms.cluster_centers_[0:2],axis=None) # assuming that there are two large peaks from the two rings
    peak1 = float(peaks[0])
    peak2 = float(peaks[1])
    # use fixed starting values if the peak1 and peak2 values are absurd
    if peak1 < data[column].min():
        peak1 = -30
    if peak2 > data[column].max():
        peak1 = 30
    estimated_peaks = {0:peak1, 1:peak2}
    
    """
    hist_data = histogram(data, binning, column)
    n = hist_data[0]
    centers = hist_data[1]
    n_left = n[centers<0]
    n_right = n[centers>=0]
    n_left_max = n_left.max()
    n_right_max = n_right.max()
    peak1 = float(centers[n == n_left_max])
    peak2 = float(centers[n == n_right_max])
    estimated_peaks = {0:peak1, 1:peak2}
    """
    

    # fit peaks
    p0 = [peak1/2, peak1, 10, peak1/2, peak2, 10]
    #p_fit, hist_data = fit_peaks(data, p0, binning=binning, column = column, hist_data = hist_data)
    p_fit, hist_data = fit_peaks(data, p0, binning=binning, column = column)
    
    
    # check order of fitted peaks (peak 1 < peak 2)
    if p_fit[1] > p_fit[4]:
        p_temp = p_fit.copy()
        p_fit[0:3]=p_temp[3:6]
        p_fit[3:6]=p_temp[0:3]
       
        
    # p_fit: amplitued_1, center_1, width_1, amplitude_2, center_2, width_2
        
    return estimated_peaks, p_fit, hist_data

def find_peak(data, binning=100, axes="y"):
    
    if axes == "y":
        column = "y_pick_rot"
    elif axes == "x":
        column = "x_pick_rot"
    elif axes == "xyz":
        column = 2
    elif axes == "z":
        column = "z"
    
    # find peak
    bandwidth = estimate_bandwidth(data[column].reshape(-1, 1), quantile=0.2, n_samples=binning)
    #print("estimated bandwidth: "+str(bandwidth))
    ms = MeanShift(bandwidth=bandwidth, bin_seeding=True)
    ms.fit(data[column].reshape(-1, 1))
    labels = ms.labels_
   # print(ms.cluster_centers_[0:2])
    peaks = np.sort(ms.cluster_centers_[0:2], axis=None)
    peak1 = float(ms.cluster_centers_[0])
    estimated_peaks = {0:peak1}
    
    # fit peaks
    p0 = [peak1/2, peak1, 40]
    
    p_fit, hist_data = fit_peak(data, p0, binning, column = column)
    #print(estimated_peaks)
    return estimated_peaks, p_fit, hist_data

def plot_peak_dist(data, hist_data, p_fit, axes="y", ax=None):
    
    if ax is None:
        ax = plt.gca()
        
    if axes == "y":
        column = "y_pick_rot"
        xlabel = "y (nm)"
    elif axes == "x":
        column = "x_pick_rot"
        xlabel = "x (nm)"
    elif axes == "z":
        column = "z"
        xlabel = "z (nm)"
    elif axes == "xyz":
        column == 1
    
    n = hist_data[0]
    bins = hist_data[1]
    
    #fig, ax = plt.subplots(figsize=(9, 5))
    binwidth = bins[1]-bins[0]
    ax.bar(bins, n, width=binwidth, color=gray)
    xlin = np.linspace(data[column].min(), data[column].max(), 1000)
    ax.plot(xlin, gaus(xlin,*p_fit[0:3]), c=red, linewidth=2)
    ax.plot(xlin, gaus(xlin,*p_fit[3:6]), c=red, linewidth=2)
    ax.set_title("Line profile",loc="left",fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Counts")  
    
    ax.text(0.5,
            0.9,
            ("Fitted Peaks:\n"
            "Peak 1 at {:.1f} nm, $\sigma$ = {:.1f} nm\n"
            "Peak 2 at {:.1f} nm, $\sigma$ = {:.1f} nm\n"
            "Distance: {:.1f} nm").format(p_fit[1],p_fit[2],p_fit[4],p_fit[5], p_fit[4]-p_fit[1]),
            horizontalalignment="center",
            verticalalignment="center",
            transform = ax.transAxes,
            fontsize=12)
    """
    ax.text(0.15,
            0.7,
            ("Estimated Peaks:\n"
            "Peak 1 at {:.1f} nm\n"
            "Peak 2 at {:.1f} nm\n"
            "Fitted Peaks:\n"
            "Peak 1 at {:.1f} nm\n"
            "Peak 2 at {:.1f} nm").format(peak1,peak2,p_fit[1],p_fit[4]),
            horizontalalignment="center",
            verticalalignment="center",
            transform = ax.transAxes,
            fontsize=12)
    """

    return ax


def plot_peak(data, hist_data, p_fit, axes="y", ax=None):
    
    if ax is None:
        ax = plt.gca()
        
    if axes == "y":
        column = "y_pick_rot"
        xlabel = "y (nm)"
    elif axes == "x":
        column = "x_pick_rot"
        xlabel = "x (nm)"
    elif axes == "z":
        column = "z"
        xlabel = "z (nm)"
    elif axes == "xyz":
        column == 1
    
    n = hist_data[0]
    bins = hist_data[1]
    
    #fig, ax = plt.subplots(figsize=(9, 5))
    binwidth = bins[1]-bins[0]
    ax.bar(bins, n, width=binwidth, color=gray)
    xlin = np.linspace(data[column].min(), data[column].max(), 1000)
    ax.plot(xlin, gaus(xlin,*p_fit[0:3]), c=red, linewidth=2)
    ax.set_title("Line profile ({})".format(axes),loc="left",fontsize=14)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Counts")  
    
    ax.text(0.5,
            0.9,
            ("Fitted Peak:\n"
            "Peak at {:.1f} nm, $\sigma$ = {:.1f} nm").format(p_fit[1],p_fit[2]),
            horizontalalignment="center",
            verticalalignment="center",
            transform = ax.transAxes,
            fontsize=12)


    return ax

def plot_locs_z_colormap(data, axes, title, fig, ax):

    # Generate data...
    if axes == "y_pick_rot":
        x = data.y_pick_rot
        y = data.x_pick_rot
        z = data.z
        ax.set_ylabel("x (nm)")
        ax.set_xlabel("y (nm)") 
    elif axes == "y":
        x = data.y
        y = data.x
        z = data.z
        ax.set_ylabel("x (nm)")
        ax.set_xlabel("y (nm)") 
    elif axes == "x_pick_rot":
        x = data.x_pick_rot
        y = data.y_pick_rot
        z = data.z
        ax.set_xlabel("x (nm)")
        ax.set_ylabel("y (nm)")
    elif axes == "x":
        x = data.x
        y = data.y
        z = data.z
        ax.set_xlabel("x (nm)")
        ax.set_ylabel("y (nm)")  
    elif axes == "z":
        x = data.x_pick_rot
        y = data.z
        z = data.z
        ax.set_xlabel("x (nm)")
        ax.set_ylabel("z (nm)")  
    
    sc = ax.scatter(x, y, s = 30, c=z, cmap='jet')
    fig.colorbar(sc, ax = ax)
    
    #ax.set_aspect('equal', adjustable='datalim')

    ax.set_title(title,loc="left",fontsize=14)
    
    return ax

def render_locs(x1, x2, locs, path, folder, pixelsize, oversampling, blur_method = 'smooth', vmin = None, vmax = None, cmap = 'hot', viewport = None, save = True):

    export_locs = locs.copy()
    
    export_locs.x 
    export_locs.y 
    
    plot_locs = export_locs.copy()
    if x1 == 'x':
        plot_locs.x = export_locs.x
    if x2 == 'x':
        plot_locs.y = export_locs.x
    if x1 == 'y':
        plot_locs.x = export_locs.y
    if x2 == 'y':
        plot_locs.y = export_locs.y
    if x1 == 'z':
        plot_locs.x = export_locs.z
    if x2 == 'z':
        plot_locs.y = export_locs.z
    


    
    if viewport is None:
        x_min = np.min(plot_locs.x)    
        x_max = np.max(plot_locs.x)
        y_min = np.min(plot_locs.y)
        y_max = np.max(plot_locs.y)
    
        frame_x = (x_max-x_min)/10
        frame_y = (y_max-y_min)/5
    
        viewport =  (y_min-frame_y, x_min-frame_x), (y_max+frame_y, x_max+frame_x)
    else:
        viewport = viewport
    
    len_x, image = render(plot_locs, viewport = viewport, oversampling=oversampling, blur_method=blur_method)
    
    img_name = "{}.png".format('image')
    img_path = os.path.join(path,folder)
    img_path_name = os.path.join(img_path,img_name)
    
    if not os.path.isdir(img_path):
        os.makedirs(img_path)


    if len(image) % 2 != 0:

        image = np.append(image,[image[-1]], axis=0)

    if len(image[0]) % 2 != 0:

        image = np.append(image,np.expand_dims(image[:,-1], axis=1), axis=1)

    if save:
        plt.imsave(img_path_name, image, vmin = vmin, vmax = vmax, cmap = cmap)



    return image
        


## Load data

In [3]:
#path = gui_fname()
path = r'X:\users\kcramer\sporPAINT\SepF\Picked_Arcs_SepF'
path = r'W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\subset'
filenames_all = load_files(path)

# filter fits that did not work properly
#  1. fit did not converge (all parameters == 0)
#  2. distance between peaks > pick width
#  3. amplitude of at least one of the peaks is <= 0
#  4. sigma of at least one of the peaks is > 30 nm or < 2nm
sigma_max = 20 # nm
sigma_min = 1 # nm

plotting = True
binning = 30 # binning for peak histogram
binning_z = 30

slice_thickness = 100 # nm
variable_slice_thickness = True # change to "False" if you want a fixed slice thickness. Otherwise the parameterslice_thickness won't be used. 
if variable_slice_thickness:
    slice_cutoff = 1.5


4 HDF5 files found.


In [4]:
# Check if some of the found hdf5 files were already analyzed?
# If yes, open ring_data dataframe with previous results.
df_arc_data = load_arc_data_df(path, "arc_data.pkl")


# Identify which files have not yet been analyzed.
if df_arc_data is not None:
    filenames = identify_new_files(filenames_all, df_arc_data)
    fov_id_start = df_arc_data['fov_id'].max()
else:
    filenames = filenames_all
    fov_id_start = 0

# Create a dictionary that saves which file was taken from which FOV and which cell types are contained (spor or veg)
# {filename_1: (FOV_id, 'spor'), filename_2: (FOV_id, 'veg'), ...}
df_fov_file_assign = identify_fov_cell_type(df_arc_data, filenames, fov_id_start)

No results of previously analyzed datasets were detected.

The cell type (spr or veg) of these files could not be determined
and thus cannot be used for further analysis:


In [5]:
for fov_id in range(int(fov_id_start)+1, int(df_fov_file_assign['fov_id'].max())+1):
    
    print('FOV ID:', fov_id)
    files_fov_id = df_fov_file_assign.loc[df_fov_file_assign['fov_id'] == fov_id]

    
    print('  spor : ', end = '')

    spor_name = files_fov_id.loc[files_fov_id['cell_type'] == 'spor']['filename']
    if spor_name.empty:
        print('--')
    else:
        print(spor_name.iloc[0])

        
    print('  veg  : ', end = '')

    veg_name = files_fov_id.loc[files_fov_id['cell_type'] == 'veg']['filename']
    if veg_name.empty:
        print('--')
    else:
        print(veg_name.iloc[0])


FOV ID: 1
  spor : Spor_arcs_SepF_230310_kcb1113_2plex_spor_fov2_500pM-r3_SepF_DP_1_drift_aligned_picked.hdf5
  veg  : veg_arcs_SepF_230310_kcb1113_2plex_spor_fov2_500pM-r3_SepF_DP_1_drift_aligned_picked.hdf5
FOV ID: 2
  spor : spor_arcs_SepF_230310_kcb1113_2plex_spor_fov1_500pM-r3_SepF_DP_2_drift_aligned_picked.hdf5
  veg  : veg_arcs_SepF_230310_kcb1113_2plex_spor_fov1_500pM-r3_SepF_DP_2_drift_aligned_picked.hdf5


## Main loop

In [8]:
# image export settings
img_format = ".png"
dpi = 100

# define colors:
blue = "#4C72B0"
orange = "#DD8452"
red = "#C44E52"
gray = "#90A8CE"

#prepare analysis folder
analysis_folder = os.path.join(path, "analysis")
if not os.path.isdir(analysis_folder):
    os.makedirs(analysis_folder)
excluded_folder = os.path.join(path, 'analysis', 'excluded_filter')
if not os.path.isdir(excluded_folder):
    os.makedirs(excluded_folder)
    
# reset old _arc.png files
# move fov_1_spor_arc.png back to analysis folder
for image_path in glob.glob(os.path.join(excluded_folder, '*_arc.png')):
    os.remove(image_path)

# remove _arc.png files from analysis folder
for image_path in glob.glob(os.path.join(analysis_folder, '*_arc.png')):
    os.remove(image_path)
    
    
# reset old _arg.png files
# move fov_1_spor_arc.png back to analysis folder
for image_path in glob.glob(os.path.join(excluded_folder, '*_zfilter.png')):
    os.remove(image_path)
    
# remove _arc.png files from analysis folder
for image_path in glob.glob(os.path.join(analysis_folder, '*_zfilter.png')):
    os.remove(image_path)
    
    
    
    
files_no_x_pick_rot = []
arc_data = []
#iterate over locs in directory:
for filename in tqdm(filenames, desc="Processing files"):
    print()
    print(filename)
    
    fov_id = df_fov_file_assign.loc[df_fov_file_assign['filename'] == filename]['fov_id'].iloc[0]
    cell_type = df_fov_file_assign.loc[df_fov_file_assign['filename'] == filename]['cell_type'].iloc[0]

    
    if np.isnan(fov_id):
        print('FOV ID is nan.')
        continue
    fov_id = int(fov_id)
    
    #load locs and convert distances from px to nm (Attention!)
    locs, info, pixelsize = load_data(os.path.join(path,filename))
    
    if locs is None:
        files_no_x_pick_rot.append(filename)
        print("File {} not loaded.".format(filename))
        continue
    
    # iterate over picks in a file 
    for pick in tqdm(np.unique(locs.group), desc="Processing picks"):
     
        # select locs from pick
        pick_locs = locs[locs.group == pick]
        
        
        ################
        # (0) filter in z: take a slice of +- slice_thickness/2
        ################
        
        estimated_peaks, r_par, hist_data = find_peak(pick_locs, binning=binning_z, axes="z")
        
        if variable_slice_thickness:
            slice_thickness = slice_cutoff * r_par[2] * 2
        
        # set up plot with gridspec
        fig0 = plt.figure(figsize=(18, 14), constrained_layout=True)
        gs0 = fig0.add_gridspec(2,3)
        fig0.suptitle(("FOV {}, {}, Pick {} - z-filter for arc analysis \n"
                          "File: {}").format(fov_id, cell_type, pick, filename), 
                      fontsize=16,
                      ha="center")

        
        # plot scatter plot for locs in pick with z colorcode
        
        
        ax1 = fig0.add_subplot(gs0[0, 2])
        ax1 = plot_locs_z_colormap(pick_locs, 
                             'z', 
                             title = 'Pick localizations',
                             fig = fig0,
                             ax = ax1)
        
        ax1.axhline(r_par[1]-(slice_thickness/2),c=blue, linewidth=2, linestyle="--")
        ax1.axhline(r_par[1]+(slice_thickness/2),c=blue, linewidth=2, linestyle="--")
        
        ax2 = fig0.add_subplot(gs0[1, 2])
        ax2 = plot_peak(pick_locs,
                       hist_data,
                       r_par,
                       axes="z",
                       ax=ax2)
        
        ax2.axvline(r_par[1]-(slice_thickness/2),c=blue, linewidth=2, linestyle="--")
        ax2.axvline(r_par[1]+(slice_thickness/2),c=blue, linewidth=2, linestyle="--")

        
        #ax1.set_xlim(ax2.get_xlim())
        #ax1.set_xlim(xmin = 0, xmax = 20)
        
        x_min, x_max = ax1.get_xlim()
        z_min, z_max = ax1.get_ylim()
        
        viewport_xz = (z_min/pixelsize, x_min/pixelsize), (z_max/pixelsize, x_max/pixelsize)
        
        frame_x = (x_max-x_min)/10
        frame_y = (pick_locs.y_pick_rot.max()-pick_locs.y_pick_rot.min())/5
        viewport_xy = ((pick_locs.y_pick_rot.min()-frame_y)/pixelsize, (x_min-frame_x)/pixelsize), ((pick_locs.y_pick_rot.max()+frame_y)/pixelsize, (x_max+frame_x)/pixelsize)

        print(viewport_xy)
        #(y_min, x_min), (y_max, x_max) = viewport
        
        # xz
        pick_locs_plot = pick_locs.copy()
        pick_locs_plot['x'] = pick_locs_plot['x_pick_rot']/pixelsize
        pick_locs_plot['y'] = pick_locs_plot['y_pick_rot']/pixelsize
        pick_locs_plot['z'] = pick_locs_plot['z']/pixelsize
        image = render_locs('x', 'z', pick_locs_plot,
                        path, 
                        'analysis', 
                        pixelsize, 
                        oversampling = 400, 
                        blur_method = 'gaussian_iso', #'gaussian_iso'
                        vmin = None,
                        vmax = None,
                        cmap = 'hot',
                        viewport = viewport_xz, 
                        save = False)
        
        ax0 = fig0.add_subplot(gs0[0, 1])
        ax0.imshow(image, aspect = 'equal', )
        ax0.invert_yaxis()
        ax0.grid(False)
        ax0.get_yaxis().set_visible(False)
        ax0.get_xaxis().set_visible(False)
        
        ax0.set_title("xz projection",loc="left",fontsize=14)
        
        # xy
        image = render_locs('x', 'y', pick_locs_plot,
                        path, 
                        'analysis', 
                        pixelsize, 
                        oversampling = 400, 
                        blur_method = 'gaussian_iso', #'gaussian_iso'
                        vmin = None,
                        vmax = None,
                        cmap = 'hot',
                        viewport = viewport_xy, 
                        save = False)
        
        ax3 = fig0.add_subplot(gs0[0, 0])
        ax3.imshow(image, aspect = 'equal', )
        ax3.invert_yaxis()
        ax3.grid(False)
        ax3.get_yaxis().set_visible(False)
        ax3.get_xaxis().set_visible(False)
        
        ax3.set_title("xy projection",loc="left",fontsize=14)

        
        
        # select locs within +- slice_thickness/2
        
        pick_locs = pick_locs[(pick_locs['z']>=r_par[1]-slice_thickness/2)&(pick_locs['z']<=r_par[1]+slice_thickness/2)]
        
        if len(pick_locs) != 0:
            
            # render locs inside of slice
            pick_locs_plot = pick_locs.copy()
            pick_locs_plot['x'] = pick_locs_plot['x_pick_rot']/pixelsize
            pick_locs_plot['y'] = pick_locs_plot['y_pick_rot']/pixelsize
            pick_locs_plot['z'] = pick_locs_plot['z']/pixelsize

            # xz
            image = render_locs('x', 'z', pick_locs_plot,
                            path, 
                            'analysis', 
                            pixelsize, 
                            oversampling = 400, 
                            blur_method = 'gaussian_iso', #'gaussian_iso'
                            vmin = None,
                            vmax = None,
                            cmap = 'hot',
                            viewport = viewport_xz,
                            save = False)

            ax4 = fig0.add_subplot(gs0[1, 1])
            ax4.imshow(image, aspect = 'equal', )
            ax4.invert_yaxis()
            ax4.grid(False)
            ax4.get_yaxis().set_visible(False)
            ax4.get_xaxis().set_visible(False)

            ax4.set_title("xz projection ({:.1f} nm z-slice)".format(slice_thickness),loc="left",fontsize=14)


            # xy
            image = render_locs('x', 'y', pick_locs_plot,
                            path, 
                            'analysis', 
                            pixelsize, 
                            oversampling = 400, 
                            blur_method = 'gaussian_iso', #'gaussian_iso'
                            vmin = None,
                            vmax = None,
                            cmap = 'hot',
                            viewport = viewport_xy, 
                            save = False)

            ax5 = fig0.add_subplot(gs0[1, 0])
            ax5.imshow(image, aspect = 'equal', )
            ax5.invert_yaxis()
            ax5.grid(False)
            ax5.get_yaxis().set_visible(False)
            ax5.get_xaxis().set_visible(False)
        
            ax5.set_title("xy projection ({:.1f} nm z-slice)".format(slice_thickness),loc="left",fontsize=14)

        
        img_zfilter_fname = "fov_{}_{}_pick_{}_zfilter".format(fov_id, cell_type, pick)
        img_zfilter_name = os.path.join(analysis_folder, img_zfilter_fname)
        fig0.savefig(img_zfilter_name+img_format, dpi=dpi, format="png")
        print(img_zfilter_name)

        plt.close(fig0)
        
        
        print(r_par)
        print(type(r_par))
        if (r_par == np.array([0,0,0])).all():
            continue
        
        
        ################
        # (1) estimate the postition of the two peaks using a histogram along the pick direction
        ################
        
        estimated_peaks, r_par, hist_data = find_peaks(pick_locs, binning=binning, axes="x")
        distance = r_par[4]-r_par[1]
        
        # set up plot with gridspec
        fig = plt.figure(figsize=(14, 16), constrained_layout=True)
        gs = fig.add_gridspec(3, 1)
        fig.suptitle(("FOV {}, {}, Pick {} - Arc analysis\n"
                          "File: {}").format(fov_id, cell_type, pick, filename), 
                      fontsize=16,
                      ha="center")
        

        
        
        # plot scatter plot for locs in pick with z colorcode
        
        
        ax1 = fig.add_subplot(gs[1, 0])
        ax1 = plot_locs_z_colormap(pick_locs, 
                             'x_pick_rot', 
                             title = 'Pick localizations',
                             fig = fig, 
                             ax = ax1)
        
        
        ax2 = fig.add_subplot(gs[2, 0])
        ax2 = plot_peak_dist(pick_locs,
                       hist_data,
                       r_par,
                       axes="x",
                       ax=ax2)
        
        
        ax1.set_xlim(ax2.get_xlim())
        #ax1.set_xlim(xmin = 0, xmax = 20)
        
        x_min, x_max = ax1.get_xlim()
        y_min, y_max = ax1.get_ylim()
        
        viewport = (y_min/pixelsize, x_min/pixelsize), (y_max/pixelsize, x_max/pixelsize)
        #print(viewport)
        #(y_min, x_min), (y_max, x_max) = viewport
        
        pick_locs_plot = pick_locs.copy()
        pick_locs_plot['x'] = pick_locs_plot['x_pick_rot']/pixelsize
        pick_locs_plot['y'] = pick_locs_plot['y_pick_rot']/pixelsize
        image = render_locs('x', 'y', pick_locs_plot,
                        path, 
                        'analysis', 
                        pixelsize, 
                        oversampling = 400, 
                        blur_method = 'gaussian_iso', #'gaussian_iso'
                        vmin = None,
                        vmax = None,
                        cmap = 'hot', 
                        viewport = viewport, 
                        save = False)
        
        ax0 = fig.add_subplot(gs[0, 0])
        ax0.imshow(image, aspect = 'equal', )
        ax0.invert_yaxis()
        ax0.grid(False)
        ax0.get_yaxis().set_visible(False)
        ax0.get_xaxis().set_visible(False)
            
        img_fname = "fov_{}_{}_pick_{}_zfilter_arc".format(fov_id, cell_type, pick)
        img_name = os.path.join(analysis_folder, img_fname)
        fig.savefig(img_name+img_format, dpi=dpi, format="png")
        print(img_name)
        plt.close(fig)
        
        
        # filter picks if the fit did not work properly
        #  1. fit did not converge (all parameters == 0)
        if all(par == 0 for par in r_par):
            filter_passed = "No"
        #  2. distance between peaks > pick width
        elif distance > info[-1]['Pick Width']*pixelsize:
            filter_passed = "No"
        #  3. amplitude of at least one of the peaks is <= 0
        elif r_par[0] <= 0 or r_par[3] <= 0:
            filter_passed = "No"
        #  4. sigma of at least one of the peaks is > 30 nm
            #sigma_max = 30 # nm
        elif r_par[2] > sigma_max or r_par[5] > sigma_max:
            filter_passed = "No"
        #  4. sigma of at least one of the peaks is < 2nm
        elif r_par[2] < sigma_min or r_par[5] < sigma_min:
            filter_passed = "No"
        else:
            filter_passed = "Yes"

        if filter_passed == 'No':
            shutil.move(os.path.join(analysis_folder, img_fname + img_format), os.path.join(excluded_folder, img_fname + img_format))
            shutil.move(os.path.join(analysis_folder, img_zfilter_fname + img_format), os.path.join(excluded_folder, img_zfilter_fname + img_format))

        
        
        arc_data.append([fov_id, #running file index
                              cell_type, # veg or spore
                              filename, #filename
                              pick, #pick number
                              r_par[1], # center 1
                              r_par[2], # sigma 1
                              r_par[0], # aplitude 1
                              r_par[4], # center 1
                              r_par[5], # sigma 1
                              r_par[3], # aplitude 1
                              distance, # distance
                              filter_passed,
                             ])
        

    """
    Plot locs
    2d gauss fit on histogram
    plot 2d Gauss
    """ 
        
        
print()
print('The following files could not be loaded.')
print('Probably they miss the x_pick_rot column:')
for filename in files_no_x_pick_rot:
    print(' -', filename)
    
    
    
# Save arc data
df_arc_data_add = pd.DataFrame(arc_data, columns=["fov_id",
                                                "cell_type",
                                                "filename",
                                                "group",
                                                "x_1",
                                                "sigma_1",
                                                "amplitude_1",
                                                "x_2",
                                                "sigma_2",
                                                "amplitude_2",
                                                "distance",
                                                "filter_passed"
                                               ])
if df_arc_data is not None:
    df_arc_data_save = pd.concat([df_arc_data, df_arc_data_add], ignore_index = True)
else:
    df_arc_data_save = df_arc_data_add
    
df_arc_data_save.to_csv("arc_data.csv")        
# Save dataframe with cell means for easy loading of data for postprocessing
df_arc_data_save.to_pickle("arc_data.pkl")


print("Data saved to CSV file in locs folder.")

Processing files:   0%|          | 0/4 [00:00<?, ?it/s]


Spor_arcs_SepF_230310_kcb1113_2plex_spor_fov2_500pM-r3_SepF_DP_1_drift_aligned_picked.hdf5


Processing picks:   0%|          | 0/3 [00:00<?, ?it/s]

((-0.03257418976380275, -0.1570808448791504), (0.19569224430964544, 0.5676120135967547))
W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\subset\analysis\fov_1_spor_pick_0_zfilter
[ 1.76125723 71.28740618 49.99145326]
<class 'numpy.ndarray'>


W:\users\reinhardt\z.software\PythonEnvironment\envs\spor_paint\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: divide by zero encountered in true_divide


...

W:\users\reinhardt\z.software\PythonEnvironment\envs\spor_paint\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: divide by zero encountered in true_divide


W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\subset\analysis\fov_2_veg_pick_16_zfilter_arc
((-0.07849410727620125, -0.508766579261193), (0.4768462430513822, 0.6911056166428786))
W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\subset\analysis\fov_2_veg_pick_17_zfilter
[ 16.51078665 -22.15903384  17.57250487]
<class 'numpy.ndarray'>
W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\subset\analysis\fov_2_veg_pick_17_zfilter_arc
((-0.08489196979082549, -0.6994582331730769), (0.5123496481088491, 0.6789325796274037))
W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\subset\analysis\fov_2_veg_pick_18_zfilter
[ 14.78863965 -43.24161504  22.42111086]
<class 'numpy.ndarray'>
W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\subset\analysis\fov_2_veg_pick_18_zfilter_arc
((-0.06092154888006357, -0.6833439917931191), (0.3671794597919171, 0.7023852586012621))
W:\users\reinhardt\z.software\Git\spor-PAINT\

Processing picks:   0%|          | 0/1 [00:00<?, ?it/s]

W:\users\reinhardt\z.software\PythonEnvironment\envs\spor_paint\lib\site-packages\ipykernel_launcher.py:181: RuntimeWarning: divide by zero encountered in true_divide


((-0.034234619140625, -0.6928262329101562), (0.2083984375, 0.695504150390625))
W:\users\reinhardt\z.software\Git\spor-PAINT\dev_sr\spor-paint\SepF\subset\analysis\fov_2_spor_pick_0_zfilter
[0 0 0]
<class 'numpy.ndarray'>

The following files could not be loaded.
Probably they miss the x_pick_rot column:
Data saved to CSV file in locs folder.
